# Laste inn i Neo4j

Starte Neo4J i Podman:
```
 podman run -d \
      -p 7474:7474 \
      -p 7687:7687 \
      -v $HOME/GIT/Tage-Politi/src-2025/OpenOwnership/Neo4j/data:/data \
      -v $HOME/GIT/Tage-Politi/src-2025/OpenOwnership/Neo4j/logs:/logs \
      -v $HOME/GIT/Tage-Politi/src-2025/OpenOwnership/Neo4j/import:/var/lib/neo4j/import \
      -e NEO4J_AUTH=neo4j/Velkommen \
      neo4j:latest
```


Hente objekter fra JSON-filen og pakke dem opp.  Dette er i essen sdet spom finnes i filen `parsa_json.py`.

In [2]:
# Standard bibliotek
from json import JSONDecoder
from functools import partial
from datetime import datetime

In [3]:
def json_parse(fileobj, 
	decoder = JSONDecoder(), 
	maxbuffer = 1048576): # 1 << 20
    
    data = ''
    for pakke in iter(partial(fileobj.read, maxbuffer), ''):
        data += pakke
        while data:
            try:
                renset = data.strip()
                objekt, index = decoder.raw_decode(renset)

                # Vi må konvertere viktige typer til "ekte" typer
                if "statementDate" in objekt:
                    try:
                        objekt["statementDate"] = datetime.strptime(objekt["statementDate"], "%Y-%m-%d")
                    except ValueError:
                        print(objekt)
                        raise
                #
                if "statementID" in objekt:
                    objekt["statementID"] = int(objekt["statementID"])
	        #
                if "interestedParty" in objekt:
                    if "describedByPersonStatement" in objekt["interestedParty"]:
                        objekt["interestedParty"]["describedByPersonStatement"] = \
                            int(objekt["interestedParty"]["describedByPersonStatement"])
                    #
                #
                if "identifiers" in objekt:
                    # Er en liste (sukk og dobbeltsukk)
                    if "id" in objekt["identifiers"][0]:
                        ids = objekt["identifiers"]
                        objekt["identifiers"][0]["id"] = \
                            int(objekt["identifiers"][0]["id"])
                    #
	        #
                if "subject" in objekt:
                    if "describedByEntityStatement" in objekt["subject"]:
                        objekt["subject"]["describedByEntityStatement"] = \
                            int(objekt["subject"]["describedByEntityStatement"])
                    #fi
                #fi                
                # Send det avgårde
                yield objekt
                data = renset[index:]
            except ValueError:
                # Trenger mer data; leser inn
                break
            #yrt
	#elihw
     #rof
#fed

Opprette forbindelsen til Neo4J

In [5]:
from neo4j import GraphDatabase
driver = GraphDatabase.driver("neo4j://localhost", auth=("neo4j", "Velkommen"))
print(driver)

Legge inn en person

Hvilken fil skal vi bruke?  Altså, hvor mange objekter?


In [6]:
FIL = "1000.json"

In [17]:

typer = {"personStatement": 0,
         "entityStatement": 0,
        "ownershipOrControlStatement": 0}
with open(FIL, "r") as fd:
    for objekt in json_parse(fd):
        typer[objekt["statementType"]] += 1
        # Vi lager noder med bare den viktige informasjonen
        print(f'StatementType: {objekt["statementType"]}')
        print(f'\tStatementID: {objekt["statementID"]}')
        if objekt["statementType"] == "personStatement":
            # Legg til personen om den ikke allerede finnes
            print(f'\tIdentifier: {objekt["identifiers"][0]["id"]}')
            cypher = f'MERGE (p:Person {{ident:{objekt["statementID"]} }})'
            print(cypher)
        elif objekt["statementType"] == "ownershipOrControlStatement":
            print(f'\tSubject: {objekt["subject"]["describedByEntityStatement"]}')
            print(f'\tPerson: {objekt["interestedParty"]["describedByPersonStatement"]}')
            cypher = f'CREATE (s:Statement {{ ident: {objekt["statementID"]} }} \
                    MERGE (p:Person {{ ident: {objekt["interestedParty"]["describedByPersonStatement"]} }} \
                    MERGE (f:Firma {{ ident: {objekt["subject"]["describedByEntityStatement"]} }} \
                    CREATE (s) - [ident:  {objekt["statementID"]} ] -> p \
                    CREATE (s) - [ident: {objekt["statementID"]}] -> f'
            print(cypher)
        elif objekt["statementType"] == "entityStatement":
            print(f'\tNavn: {objekt["name"]}')
            cypher = f'MERGE (f:Firma {{ ident: {objekt["statementID"]}, name: \"{objekt["name"]}\" }})'
            print(cypher)
        else:
            assert(0 == "Ukjent type")
        #fi
    print(typer)
#



StatementType: entityStatement
	StatementID: 100019692634382761
	Navn: DEN SELVEJENDE INSTITUTION SØHOLM
MERGE (f:Firma { ident: 100019692634382761, name: "DEN SELVEJENDE INSTITUTION SØHOLM" })
StatementType: entityStatement
	StatementID: 10004474695595307041
	Navn: SæbeGrossisten ApS
MERGE (f:Firma { ident: 10004474695595307041, name: "SæbeGrossisten ApS" })
StatementType: personStatement
	StatementID: 10008819103465301157
	Identifier: 4003890979
MERGE (p:Person {ident:10008819103465301157 })
StatementType: ownershipOrControlStatement
	StatementID: 10011268000246566669
	Subject: 84347118211408559
	Person: 3733202849081635896
CREATE (s:Statement { ident: 10011268000246566669 }                     MERGE (p:Person { ident: 3733202849081635896 }                     MERGE (f:Firma { ident: 84347118211408559 }                     CREATE (s) - [ident:  10011268000246566669 ] -> p                     CREATE (s) - [ident: 10011268000246566669] -> f
StatementType: entityStatement
	StatementID: 